In [26]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from openai import OpenAI
import json
import time
import re
from tqdm import tqdm
import asyncio
from typing import List, Dict, Any, Tuple, Union

from core.features.assessments.prompts import SIMPLE_ASSIGNMENT_CHECK_PROMPT
from core.features.utils import add_dicts, calculate_cost_gpt4_turbo
from core.features.provider import creator, async_creator, text_model_defaults

from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

- marks = 6

- feedback = you are good

In [9]:
import json

a = [23, [34, 45]]

obj = json.dumps(a)

obj

'[23, [34, 45]]'

In [7]:
q = "#q1\n"

q.replace('q', 'a')


for i in range(10):
    for j in range(i):
        if j > 5:
            print(j)
            break



6
6
6


In [32]:

def question_or_answer_cell(s):

    """
    Function to check the correct tag for a question or an answer cell.
    Must match - "#q<some + integer>" or "#a<some + integer>"
    """

    pattern = r"^#[aq]\d+"
    s = s.strip()
    return bool(re.match(pattern, s))


def qna_extraction(file_path):
    """
    Function to extracts question and answers from the jupyter notebook
    cells with #qn tag and #an tag
    return the dict of all the questions and answers 
    """

    if file_path.endswith(".ipynb"):
        with open(file_path, "r") as f:
            nb = json.load(f)
    else:
        ValueError("File type not supported. Must be a valid Jupyter Notebook file.")

    cells = nb['cells']
    qnas = {}
    for cell in cells:
        if question_or_answer_cell(cell['source'][0]):
            qnas[cell['source'][0].strip('#\n')] = "".join(cell['source'][1:])  
    return qnas


async def assessment_check(qna_dict):

    """
    Main function to check the solution of the answer
    provide obtained marks and feedback
    """

    conversation = [{"role": "system", "content": SIMPLE_ASSIGNMENT_CHECK_PROMPT}]
    user_input = f"""
    
    {qna_dict}

    """
    user_message = {'role': 'user', 'content': user_input}
    conversation.append(user_message)

    response = await async_creator(
        **text_model_defaults,
        messages=conversation
    )    
    response = response.model_dump()
    output = response["choices"][0]["message"]["content"]
    total_usage = response["usage"]
    return {"output": output, "total_usage": total_usage}



def notebook_check(qnas, marks):

    """
    This function takes the set of questions and answers and provide a dict with remarks of all questions
    """

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    remarks = {}
    for i in range(1, len(marks)+1):
        qna_dict = {
            "question": qnas[f"q{i}"],
            "solution": qnas[f"a{i}"],
            "max_marks": marks[f"q{i}"]
        }
        result = assessment_check(qna_dict=qna_dict)
        output = json.loads(result['output'])
        usage = result['total_usage']

        total_usage = add_dicts(total_usage, usage)
        
        qna_dict.update(output)
        remarks[f"q{i}"] = qna_dict

    return remarks, total_usage

In [39]:
def batch_assessment_check(path, marks=None, remarks_filename="remarks.json"):
    """
    This function is the main batch processing function for assessment
    Takes path of folder or notebook, check all the questions of all the notebooks and provide a combined json for remarks

    path: can be jupyter notebook path or a folder full of jupyter notebooks
    marks: a dict with questions(qn) and max_marks mapping
    """
    
    total_tokens = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    final_remarks = {}

    remarks_path = "/"
    if os.path.isdir(path):
        print("GOT A DIR")
        file_paths = [os.path.join(path, file_path) for file_path in os.listdir(path)[:2] if file_path.endswith('.ipynb')]
        print(f"GOT {len(file_paths)} notebooks")
        files_and_qnas = {file_path: qna_extraction(file_path) for file_path in file_paths}
        print("EXTRACTED QNAS")
        remarks_path = os.path.join(path, 'remarks')
        os.makedirs(remarks_path, exist_ok=True)
        print("CREATED REMARKS FOLDER")

    elif path.endswith('.ipynb'):
        print("GOT A NOTEBOOK")
        qnas = qna_extraction(file_path=path)
        print("EXTRACTED QNAS")
        files_and_qnas = {path: qnas}
        remarks_path = os.path.dirname(path)

    for file, qna in tqdm(files_and_qnas.items()):
        remarks, tokens = notebook_check(qna, marks)
        # remarks_dict = {f"q{i+1}": remark for i, remark in enumerate(remarks)}
        # print(remarks)
        final_remarks[os.path.basename(file)] = remarks
        total_tokens = add_dicts(total_tokens, tokens)

    print("PROCESSING COMPLETE")
    # saving the remarks
    json_remarks = json.dumps(final_remarks, indent=4)
    with open(os.path.join(remarks_path, remarks_filename), 'w') as f:
        f.write(json_remarks)

    print("DONE!")

    return final_remarks, total_tokens 

In [34]:
# assignment 1 test cases

marks = {
    "q1": 1,
    "q2": 1,
    "q3": 1,
    "q4": 4,
    "q5": 5,
    "q6": 2,
    "q7": 2,
    "q8": 8,
    "q9": 1
}

ass1_folder = r'core\features\assessments\ass1_notebooks'
final_remarks, total_tokens = batch_assessment_check(ass1_folder, marks = marks, remarks_filename='remarks.json')

print(calculate_cost_gpt4_turbo(total_tokens))

GOT A DIR
GOT 2 notebooks
EXTRACTED QNAS
CREATED REMARKS FOLDER


100%|██████████| 2/2 [01:05<00:00, 32.65s/it]

PROCESSING COMPLETE
DONE!


In [ ]:
# l = [2,3,4,5]
# for i in l:
#     print(i)
#     l.insert(1, '2')
#     print(l)

# print(l)

In [12]:
import uuid

print(str(uuid.uuid4()))
print(str(uuid.uuid4()))
print(str(uuid.uuid4()))
print(str(uuid.uuid4()))

598eba1b-90bf-4427-a2e8-09efeda228d5
3a8cc642-e35c-455f-9b3e-a1eb29eec638
8f225da0-f0e4-40d0-963f-8e548456f1a7
47eae615-7add-48cc-9bcc-7f7e6a53e8d6


In [40]:
# assignment 2 test cases

marks = {
    "q1": 3,
    "q2": 5,
    "q3": 10,
    "q4": 5,
    "q5": 2
}

ass2_folder = r'core\features\assessments\ass2_notebooks'
final_remarks, total_tokens = batch_assessment_check(ass2_folder, marks = marks, remarks_filename='remarks2.json')

GOT A DIR
GOT 2 notebooks
EXTRACTED QNAS
CREATED REMARKS FOLDER


100%|██████████| 2/2 [00:38<00:00, 19.15s/it]

PROCESSING COMPLETE
DONE!


In [38]:
calculate_cost_gpt4_turbo(total_tokens)

{'usd': 0.10194, 'inr': 8.4865}

In [44]:
a = {str({1:2}): 2}

a

{'{1: 2}': 2}